In [49]:
import pandas as pd
import yaml
import pandas as pd
from pyprojroot import here
from collections import Counter
import os
import plotly.express as px

In [50]:
with open(here("configs/config.yml")) as cfg:
    cfg = yaml.load(cfg, Loader=yaml.FullLoader)
questions_df = pd.read_excel(os.path.join(
    here(cfg["eval_questions_dir"]), cfg["eval_file_name"]))
final_df = pd.DataFrame(columns=["best_scorer", "num_top_appearance"])

**Check for Nan values**

In [3]:
print(questions_df["langchain_token_mmr_score"].isna().sum())
print(questions_df[questions_df["langchain_token_mmr_score"].isna()].index)

0
Index([], dtype='int64')


In [4]:
questions_df.columns

Index(['source', 'question', 'correct_answer', 'langchain_token_mmr_result',
       'langchain_token_mmr_inference_time',
       'langchain_recursive_similarity_result',
       'langchain_recursive_similarity_inference_time',
       'langchain_recursive_mmr_result',
       'langchain_recursive_mmr_inference_time',
       'llama_index_sentence_retrieval_result',
       'llama_index_sentence_retrieval_inference_time',
       'llama_index_auto_merging_retrieval_result',
       'llama_index_auto_merging_retrieval_inference_time',
       'langchain_recursive_similarity_score', 'langchain_recursive_mmr_score',
       'llama_index_sentence_retrieval_score',
       'llama_index_auto_merging_retrieval_score', 'lowest_score',
       'highest_score', 'langchain_token_mmr_score',
       'langchain_token_mmr_result_score', 'langchain_token_mmr_score_score'],
      dtype='object')

**Compute and print total scores (from 40):**

In [5]:
langchain_token_mmr_total_score = sum(questions_df["langchain_token_mmr_score"])
langchain_similarity_total_score = sum(questions_df["langchain_recursive_similarity_score"])
langchain_mmr_total_score = sum(questions_df["langchain_recursive_mmr_score"])
llama_index_sentence_retrieval_score = sum(questions_df["llama_index_sentence_retrieval_score"])
llama_index_auto_merging_retrieval_total_score = sum(questions_df["llama_index_auto_merging_retrieval_score"])
print(f"Total scores:\n\
    langchain_token_mmr: {round(langchain_token_mmr_total_score, 2)}, \n\
    langchain_recursive_similarity: {round(langchain_similarity_total_score, 2)},\n\
    langchain_recursive_mmr: {round(langchain_mmr_total_score, 2)},\n\
    llama_index_sentence_retrieval: {round(llama_index_sentence_retrieval_score, 2)},\n\
    llama_index_auto_merging_retrieval: {round(llama_index_auto_merging_retrieval_total_score, 2)}")
scorer_list = [langchain_token_mmr_total_score, langchain_similarity_total_score, langchain_mmr_total_score, llama_index_sentence_retrieval_score, llama_index_auto_merging_retrieval_total_score]

Total scores:
    langchain_token_mmr: 34.1, 
    langchain_recursive_similarity: 34.3,
    langchain_recursive_mmr: 33.5,
    llama_index_sentence_retrieval: 35.4,
    llama_index_auto_merging_retrieval: 28.9


* Best overall performance:  
* Wrost overall performance: 

In [51]:
final_df["best_scorer"] = scorer_list

------------------------------------------

In [7]:
score_df = questions_df[[
    'langchain_token_mmr_score',
    'langchain_recursive_similarity_score',
    'langchain_recursive_mmr_score',
    'llama_index_sentence_retrieval_score',
    'llama_index_auto_merging_retrieval_score']]
max_cols = score_df.apply(lambda x: x[x == x.max()].index.tolist(), axis=1)
max_cols_list = max_cols.to_list()

min_cols = score_df.apply(lambda x: x[x == x.min()].index.tolist(), axis=1)
min_cols_list = min_cols.to_list()

**Count the number of times that each technique was among the highest scorers:**

In [8]:
# Count the number of times each column name appears in the list
max_cols_count = Counter([col for row in max_cols_list for col in row])
max_cols_count
top_appearance_list = list(max_cols_count.values())

In [9]:
max_cols_count

Counter({'llama_index_sentence_retrieval_score': 31,
         'langchain_recursive_similarity_score': 31,
         'langchain_token_mmr_score': 22,
         'langchain_recursive_mmr_score': 12,
         'llama_index_auto_merging_retrieval_score': 8})

In [52]:
final_df["num_top_appearance"] = top_appearance_list

**Count the number of times that each technique was among the lowest scorers:**

In [11]:
# Count the number of times each column name appears in the list
min_cols_count = Counter([col for row in min_cols_list for col in row])
min_cols_count

Counter({'llama_index_auto_merging_retrieval_score': 30,
         'langchain_recursive_mmr_score': 16,
         'langchain_token_mmr_score': 10,
         'langchain_recursive_similarity_score': 7,
         'llama_index_sentence_retrieval_score': 6})

---------------------------------------

In [12]:
def count_number_of_zero_scores(df, column_name):
    return len(df[df[column_name]==0])

langchain_token_mmr_total_zero_score = count_number_of_zero_scores(questions_df, "langchain_token_mmr_score")
langchain_recursive_similarity_total_zero_score = count_number_of_zero_scores(questions_df, "langchain_recursive_similarity_score")
langchain_recursive_mmr_total_zero_score = count_number_of_zero_scores(questions_df, "langchain_recursive_mmr_score")
llama_index_sentence_retrieval_zero_score = count_number_of_zero_scores(questions_df, "llama_index_sentence_retrieval_score")
llama_index_auto_merging_retrieval_total_zero_score = count_number_of_zero_scores(questions_df, "llama_index_auto_merging_retrieval_score")
print(f"Total zero scores:\n\
    langchain_token_mmr: {langchain_token_mmr_total_zero_score},\n\
    langchain_recursive_similarity: {langchain_recursive_similarity_total_zero_score},\n\
    langchain_recursive_mmr: {langchain_recursive_mmr_total_zero_score},\n\
    llama_index_sentence_retrieval: {llama_index_sentence_retrieval_zero_score},\n\
    llama_index_auto_merging_retrieval: {llama_index_auto_merging_retrieval_total_zero_score}")

Total zero scores:
    langchain_token_mmr: 0,
    langchain_recursive_similarity: 0,
    langchain_recursive_mmr: 1,
    llama_index_sentence_retrieval: 0,
    llama_index_auto_merging_retrieval: 2


In [13]:
# all_highest_scores = [score.strip() for sublist in questions_df['highest_score'].str.split(', ') for score in sublist]
# highest_score_counts = Counter(all_highest_scores)
# highest_score_counts

In [14]:
# all_lowest_scores = [score.strip() for sublist in questions_df['lowest_score'].str.split(', ') for score in sublist]
# lowest_score_counts = Counter(all_lowest_scores)
# lowest_score_counts

------------------------------------

**Analysis based on each document**

In [15]:
langchain_token_mmr_df = questions_df.pivot_table(index='source', columns=None, values='langchain_token_mmr_score', aggfunc='sum')
langchain_recursive_similarity_df = questions_df.pivot_table(index='source', columns=None, values='langchain_recursive_similarity_score', aggfunc='sum')
langchain_recursive_mmr_df = questions_df.pivot_table(index='source', columns=None, values='langchain_recursive_mmr_score', aggfunc='sum')
llama_index_sentence_df = questions_df.pivot_table(index='source', columns=None, values='llama_index_sentence_retrieval_score', aggfunc='sum')
llama_index_auto_merging_df = questions_df.pivot_table(index='source', columns=None, values='llama_index_auto_merging_retrieval_score', aggfunc='sum')

# Concatenate the DataFrames into a single DataFrame
result_df = pd.concat([langchain_token_mmr_df, langchain_recursive_similarity_df, langchain_recursive_mmr_df, llama_index_sentence_df, llama_index_auto_merging_df], axis=1)

# Rename the columns to match your desired output
result_df.columns = [
    'langchain_token_mmr_score',
    'langchain_recursive_similarity_score',
    'langchain_recursive_mmr_score',
    'llama_index_sentence_retrieval_score',
    'llama_index_auto_merging_retrieval_score'
    ]

# Display the result
display(result_df)

,langchain_token_mmr_score,langchain_recursive_similarity_score,langchain_recursive_mmr_score,llama_index_sentence_retrieval_score,llama_index_auto_merging_retrieval_score
source,,,,,
SegmentAnything paper,12.1,11.5,11.3,12.1,9.7
product scpecification,3.5,4.2,3.9,4.2,4.0
stories,4.5,3.7,4.2,4.7,3.0
technical support,4.5,4.8,4.9,4.5,4.5
vision transformer paper,9.5,10.1,9.2,9.9,7.7


**Average score per document (normalized)**

In [16]:
print("Normalized average score per document:")
num_questions = questions_df[["source", "question"]].groupby("source").count().values.tolist()
num_questions = [val for sublist in num_questions for val in sublist]
mean_doc_score = result_df.mean(axis=1).values.tolist()
result = [round(x / y, 2) for x, y in zip(mean_doc_score, num_questions)]
result

Normalized average score per document:


[0.81, 0.79, 0.8, 0.93, 0.84]

* SegmentAnything paper: (2 column scientific paper) 
    - Best scorer: 
    - Wrost scorer:
    - Normalized average score:
* product scpecification:
    - Best scorer: 
    - Wrost scorer:
    - Normalized average score:
* stories:
    - Best scorer: 
    - Wrost scorer:
    - Normalized average score:
* technical support:
    - Best scorer: 
    - Wrost scorer:
    - Normalized average score:
* vision transformer paper: (1 coulmn scientific paper)
    - Best scorer: 
    - Wrost scorer:
    - Normalized average score:
-----------------------------

**Total inference time**

In [17]:
questions_df[[
    "langchain_token_mmr_inference_time",
    "langchain_recursive_mmr_inference_time",
    "langchain_recursive_similarity_inference_time",
    "llama_index_sentence_retrieval_inference_time",
    "llama_index_auto_merging_retrieval_inference_time"
    ]].sum()

langchain_token_mmr_inference_time                    98.86
langchain_recursive_mmr_inference_time                96.50
langchain_recursive_similarity_inference_time         90.60
llama_index_sentence_retrieval_inference_time        121.71
llama_index_auto_merging_retrieval_inference_time     65.47
dtype: float64

**Average inference time**

In [18]:
questions_df[[
    "langchain_token_mmr_inference_time",
    "langchain_recursive_mmr_inference_time",
    "langchain_recursive_similarity_inference_time",
    "llama_index_sentence_retrieval_inference_time",
    "llama_index_auto_merging_retrieval_inference_time"
    ]].mean()

langchain_token_mmr_inference_time                   2.47150
langchain_recursive_mmr_inference_time               2.41250
langchain_recursive_similarity_inference_time        2.26500
llama_index_sentence_retrieval_inference_time        3.04275
llama_index_auto_merging_retrieval_inference_time    1.63675
dtype: float64

* Fastest: llama_index_auto_merging_retrieval_inference_time
* slowest: llama_index_sentence_retrieval_inference_time

-------------------------

**Check the zero scored questions**

In [19]:
print("llama_index_auto_merging_retrieval:")
display(questions_df[questions_df["llama_index_auto_merging_retrieval_score"]==0])
print("llama_index_sentence_retrieval:")
display(questions_df[questions_df["llama_index_sentence_retrieval_score"]==0])
print("langchain_token_mmr:")
display(questions_df[questions_df["langchain_token_mmr_score"]==0])
print("langchain_recursive_mmr:")
display(questions_df[questions_df["langchain_recursive_mmr_score"]==0])
print("langchain_recursive_similarity:")
display(questions_df[questions_df["langchain_recursive_similarity_score"]==0])

llama_index_auto_merging_retrieval:


,source,question,correct_answer,langchain_token_mmr_result,langchain_token_mmr_inference_time,langchain_recursive_similarity_result,langchain_recursive_similarity_inference_time,langchain_recursive_mmr_result,langchain_recursive_mmr_inference_time,llama_index_sentence_retrieval_result,...,llama_index_auto_merging_retrieval_inference_time,langchain_recursive_similarity_score,langchain_recursive_mmr_score,llama_index_sentence_retrieval_score,llama_index_auto_merging_retrieval_score,lowest_score,highest_score,langchain_token_mmr_score,langchain_token_mmr_result_score,langchain_token_mmr_score_score
0,stories,"Who are Amarok, Fred, and Lily?",Amarok the Lone Wolf:\n\nAmarok is a lone wolf...,Amarok is a lone wolf who lived in the Alaskan...,1.65,"I'm sorry, but I couldn't find any information...",1.99,Amarok is a lone wolf who embarked on a journe...,2.16,"Amarok, Fred, and Lily are characters in diffe...",...,2.22,0.5,0.8,1.0,0.0,llama_index_auto_merging_retrieval,llama_index_sentence_retrieval,0.9,0.8,NaN
13,product scpecification,How much does CubeTriangle Kappa Portable Spea...,2000,The CubeTriangle Kappa Portable Speaker costs ...,0.43,The CubeTriangle Kappa Portable Speaker is pri...,0.45,The CubeTriangle Kappa Portable Speaker is not...,1.10,The price of the CubeTriangle Kappa Portable S...,...,0.98,0.3,0.0,0.2,0.0,llama_index_auto_merging_retrieval,llama_index_sentence_retrieval,0.3,0.6,NaN


llama_index_sentence_retrieval:


,source,question,correct_answer,langchain_token_mmr_result,langchain_token_mmr_inference_time,langchain_recursive_similarity_result,langchain_recursive_similarity_inference_time,langchain_recursive_mmr_result,langchain_recursive_mmr_inference_time,llama_index_sentence_retrieval_result,...,llama_index_auto_merging_retrieval_inference_time,langchain_recursive_similarity_score,langchain_recursive_mmr_score,llama_index_sentence_retrieval_score,llama_index_auto_merging_retrieval_score,lowest_score,highest_score,langchain_token_mmr_score,langchain_token_mmr_result_score,langchain_token_mmr_score_score


langchain_token_mmr:


,source,question,correct_answer,langchain_token_mmr_result,langchain_token_mmr_inference_time,langchain_recursive_similarity_result,langchain_recursive_similarity_inference_time,langchain_recursive_mmr_result,langchain_recursive_mmr_inference_time,llama_index_sentence_retrieval_result,...,llama_index_auto_merging_retrieval_inference_time,langchain_recursive_similarity_score,langchain_recursive_mmr_score,llama_index_sentence_retrieval_score,llama_index_auto_merging_retrieval_score,lowest_score,highest_score,langchain_token_mmr_score,langchain_token_mmr_result_score,langchain_token_mmr_score_score


langchain_recursive_mmr:


,source,question,correct_answer,langchain_token_mmr_result,langchain_token_mmr_inference_time,langchain_recursive_similarity_result,langchain_recursive_similarity_inference_time,langchain_recursive_mmr_result,langchain_recursive_mmr_inference_time,llama_index_sentence_retrieval_result,...,llama_index_auto_merging_retrieval_inference_time,langchain_recursive_similarity_score,langchain_recursive_mmr_score,llama_index_sentence_retrieval_score,llama_index_auto_merging_retrieval_score,lowest_score,highest_score,langchain_token_mmr_score,langchain_token_mmr_result_score,langchain_token_mmr_score_score
13,product scpecification,How much does CubeTriangle Kappa Portable Spea...,2000,The CubeTriangle Kappa Portable Speaker costs ...,0.43,The CubeTriangle Kappa Portable Speaker is pri...,0.45,The CubeTriangle Kappa Portable Speaker is not...,1.1,The price of the CubeTriangle Kappa Portable S...,...,0.98,0.3,0.0,0.2,0.0,llama_index_auto_merging_retrieval,llama_index_sentence_retrieval,0.3,0.6,NaN


langchain_recursive_similarity:


,source,question,correct_answer,langchain_token_mmr_result,langchain_token_mmr_inference_time,langchain_recursive_similarity_result,langchain_recursive_similarity_inference_time,langchain_recursive_mmr_result,langchain_recursive_mmr_inference_time,llama_index_sentence_retrieval_result,...,llama_index_auto_merging_retrieval_inference_time,langchain_recursive_similarity_score,langchain_recursive_mmr_score,llama_index_sentence_retrieval_score,llama_index_auto_merging_retrieval_score,lowest_score,highest_score,langchain_token_mmr_score,langchain_token_mmr_result_score,langchain_token_mmr_score_score


In [20]:
for i in [0, 38, 13, 30, 13]:
    print(questions_df[questions_df.index==i]["question"].values[0])

Who are Amarok, Fred, and Lily?
What ablations were performed on the 23 dataset suite with the single center point prompt protocol, and what were the results?
How much does CubeTriangle Kappa Portable Speaker cost?
How was the data engine used to collect the SA-1B mask dataset, and what were the three stages of the data collection process?
How much does CubeTriangle Kappa Portable Speaker cost?


In [53]:
final_df.index = ['langchain_token_mmr', 'langchain_recursive_mmr', 'langchain_recursive_similarity', 'llama_index_sentence_retrieval', 'llama_index_auto_merging_retrieval']
final_df = final_df.reset_index().rename(columns={'index': 'method'})


In [104]:
color_sequence = px.colors.qualitative.Plotly

# Create the scatter plot
fig = px.scatter(
    final_df,
    x='method',
    y='best_scorer',
    size='num_top_appearance',
    hover_name='method',
    color='method',  # Assign colors based on the 'system' column
    color_discrete_sequence=color_sequence,  # Use the defined color sequence
    title='<b>Best Scorer by method<b>'
)

# Customize the layout
fig.update_layout(
    title_font=dict(family='Arial, sans-serif', size=24, color='black'),
    title_x=0.5,  # Center the title
    paper_bgcolor='white',  # Background color for the outer area
    plot_bgcolor='white',  # Background color for the plot area
    xaxis=dict(
        title='<b>Methods<b>',
        title_font=dict(size=24),
        tickangle=45,  # Rotate labels
        tickfont=dict(family='Arial, sans-serif', size=18, color='black'),
        showgrid=True  # Hide gridlines
    ),
    yaxis=dict(
        title='<b>Best Scorer<b>',
        title_font=dict(family='Arial, sans-serif', size=18, color='black'),
        tickfont=dict(family='Arial, sans-serif', size=16, color='black'),
        showgrid=True,  # Show gridlines
        gridcolor='lightgrey'  # Gridline color
    ),
    legend=dict(
        title='Methods',
        font=dict(family='Arial, sans-serif', size=16, color='black',),
        bgcolor='lightgrey',  # Background color for the legend
        bordercolor='black',  # Border color for the legend
        borderwidth=1
    ),
    margin=dict(l=60, r=60, t=60, b=60),  # Adjust margins to fit labels
    width=1800,  # Width of the entire plot image
    height=600  # Height of the entire plot image
)

# Customize marker appearance
fig.update_traces(
    marker=dict(
        line=dict(width=3, color='darkgrey')  # Border line for markers
    ),
    selector=dict(mode='markers')
)

# Show the plot
fig.show()